# Taking frames from video clips using CV2

##### Matthew Johnson, 2019


TODO:
- Cropping:
    - Top piece - Classify as H2H, Squads, or Solo.
    - Bottom piece - Classify as Verticals, WR Post, HB Pitch, Fake FG, etc.

In [75]:
import cv2
import os
import pandas as pd


save_path = 'saved_frames'
if not os.path.isdir(save_path):
    os.mkdir(save_path)


def extract_final_frame(video_file):
    
    folder_path = '/Volumes/PS4 MyPassp/PS4/SHARE/Video Clips/Madden NFL 19'
    #video_file = 'Madden NFL 19_20190202233415.mp4'
    video_path = os.path.join(folder_path, video_file)

    # Opens the Video file
    cap= cv2.VideoCapture(video_path)
    i=0
    frame_holder = None
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        if ret:
            frame_holder = frame  
        else:
            break    
        i+=1

    save_prefix = video_file.rstrip('.mp4')
    save_filename = f'{save_path}/{save_prefix}-frame{i}.jpg'
    cv2.imwrite(save_filename, frame_holder)
    cap.release()
    cv2.destroyAllWindows()
    
    # 
    suffix = video_file.rstrip('.mp4').split('_')[-1]
    entry = {'video_file': video_file,
              'cap_file': save_filename,
              'date': suffix
             }
    return entry


def get_todo_filenames():
    
    clip_file = 'all_clip_names.csv'
    if os.path.isfile(clip_file):
        all_clips = pd.read_csv(clip_file)
        todo_files = all_clips[ all_clips['capped'] == 0]['filename'].values
    else:
        source_path = '/Volumes/PS4 MyPassp/PS4/SHARE/Video Clips/Madden NFL 19'
        todo_files = [f for f in os.listdir(source_path) if f.endswith('.mp4')]
        
    return todo_files


def combined_dataframes(new_df):
    old_df = pd.read_csv('all_clip_names.csv')
    old_df = old_df[ old_df['capped'] == 1]
    
    cdf = pd.concat([old_df, new_df], axis=0)
    # rename when confirmed working
    cdf.to_csv('all_clip_names_TEST.csv')

# ...

In [76]:
#
save_name = 'may19_capped_videos'

filenames = get_todo_filenames()
stop_idx = min(100, len(filenames))
print(f'{len(filenames)} video files available...')
data_entries = [extract_final_frame(file) for file in filenames[:stop_idx]]
df = pd.DataFrame(data_entries)

if not os.path.isfile(save_name):
    df.to_csv(save_name + '.csv')
else: 
    for i in range(100):
        if not os.path.isfile(save_name + f'_{i}.csv')
            df.to_csv(save_name + f'_{i}.csv')
            break
#
df2 = pd.DataFrame({'filename': filenames})
df2['date'] = df2['filename'].str.split('Madden NFL 19_').apply(lambda x: x[1]).str[:8]
df2['date'] = pd.to_datetime(df2['date'], format='%Y%m%d', errors='coerce')
df2['capped'] = df2['filename'].apply(lambda x: x in df['video_file'].values)*1
df2.sort_values('date', ascending=False, inplace=True)
df2.reset_index(drop=True, inplace=True)